# Raw data
--------------
## first check the raw data

In [4]:
# check how many lines
! wc -l data/emojitweets-01-04-2018.txt

18866900 data/emojitweets-01-04-2018.txt


In [2]:
# check how many words
! wc -w data/emojitweets-01-04-2018.txt

209430000 data/emojitweets-01-04-2018.txt


In [3]:
# On average the sentense are have 11 words
209430000/18866900

11.100392751326398

## Train - test - validation split

In [6]:
# split on 80-10-10
18866900 * 0.8, 18866900 * 0.1, 18866900 * 0.2

(15093520.0, 1886690.0, 3773380.0)

In [8]:
! head -15093520 data/emojitweets-01-04-2018.txt >>emoji_train.txt


In [14]:
! (tail -3773380 data/emojitweets-01-04-2018.txt | head -1886690) >> emoji_val.txt

tail: error writing 'standard output': Broken pipe


In [10]:
! tail -1886690 data/emojitweets-01-04-2018.txt >> emoji_test.txt

In [1]:
# generate small_sample
! head -1509352 data/emoji_train.txt >> data/emoji_ss.txt

# Word Vectorize

In [1]:
# Python program to generate word vectors using Word2Vec 
  
# importing all necessary modules 
from nltk.tokenize import sent_tokenize, word_tokenize 
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

import emoji
import regex

import pandas as pd
import numpy as np
import pickle


In [2]:
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer


In [131]:
# sample = open("emoji_1M.txt", "r") 
sample = open("emoji_ss.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
f = s.replace("\n", " ") 

In [137]:
s[:500]

"Squad arriving for Game 2 🚀\nDude is like 5’8 140 pounds his dick was long and strong(always the little dudes carrying the 🍆) \U0001f92a🙃\nFOLLOWERS👇\nI CANT BREATIUHW 💀💀💀\n2️⃣4️⃣ hours 'til our schedule drops!\nNEW || Zach &amp; Jack at Limelight tonight! (April 17) ©️nvmbesson\nI’m live at Milk Boy studio the home of  making some hits 💪🏾\nI am SO scared of birds🤧\nThat’s me 😂😂😂😂😂😂\nMy heart is so full rn 💖💖\nThis is one of my favorite songs to sing in this episode ❤️\nTook my goat to get groomed for the first tim"

In [138]:
# sample = open("emoji_1M.txt", "r") 
sample = open("emoji_ss.txt", "r") 
s = sample.read() 
  
# Replaces escape character with space 
# f = s.replace("\n", " ") 
f = s


# replace all 
notes = [",", ".", "/'", "(" ,")", "'", '/"', ":", ";", "/", "(", ")", "“", "”", "-", "+", "#", "…", "!"]
for note in notes:
    f = f.replace(note, "")
  
data = [] 
  

In [130]:
len(f[0])

1

In [114]:
# move to helper
def split_count(text):

    emoji_list = []
    data = regex.findall(r'\X', text)
    for word in data:
        if any(char in emoji.UNICODE_EMOJI for char in word):
            emoji_list.append(word)

    return emoji_list

### Small Sample

In [4]:
# sample = open("../emoji_100k.txt", "r") 
# s_ss = sample.read() 
  
# # Replaces escape character with space
# f_ss = s_ss.lower().replace("\n", " ")

# # replace all 
# signs = [",", ".", "(" ,")", "'", '/"', ":", ";", "/", "(", ")"]
# for sign in signs:
#     f_ss = f_ss.replace(sign, "")
  
# data = [] 

In [139]:
# f = f_ss
# iterate through each sentence in the file 
for i in sent_tokenize(f): 
    temp = [] 
      
    # tokenize the sentence into words 
    for j in word_tokenize(i): 
        temp.append(j.lower()) 
  
    data.append(temp) 

In [140]:
# import stopwords
stop = set(stopwords.words('english'))



In [141]:
# remove stop words and snowball stemmer
out = [[word for word in words if word not in stop] for words in data]
snowball = SnowballStemmer('english')
docs_snowball = [[snowball.stem(word) for word in words] for words in out]

In [142]:
len(docs_snowball)

79777

In [144]:
len(docs_snowball[0])

266

save emoji list

In [10]:
# emoji_list = list(emoji_set)
# with open("emoji_list.csv", 'w', newline='') as myfile:
#      w = csv.writer(myfile, quoting=csv.QUOTE_ALL)
#      w.writerow(emoji_list)

4358

In [ ]:
# with open("emoji_list.csv", 'r', newline='') as myfile:
#      emoji_list = csv.reader(myfile, quoting=csv.QUOTE_ALL)

# CBOW model

In [145]:
# Create CBOW model 
model1 = gensim.models.Word2Vec(docs_snowball, 
                                min_count = 10,  
                                size = 300, 
                                window = 16,
                                workers= 8) 

In [15]:
# Save CBOW model
import pickle
with open('cbow_ss.pkl', 'wb') as f:
    pickle.dump(model1, f)

In [14]:
model1.

{'vocab': 19961500,
 'vectors': 31938400,
 'syn1neg': 31938400,
 'total': 83838300}

In [10]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)
emoji_list = val_df["emoji"].values.tolist()

In [11]:
# move to helper
def emoji_prodictor(word, model):
    sim = model.most_similar(word,topn=75)
    for i in sim:
        if i[0] in emoji_list:
            return i[0]
    return word

In [12]:
# move to helper
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out
    
    
s1 = "let's go grocery and buy some apple"
word_pipeline(s1)

['let', 'go', 'groceri', 'buy', 'appl']

In [13]:
s1 = "let's go grocery and buy some apple"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model1))


🤷‍♀️
🤦🏾‍♀️
🛒
💸
👐🏽


In [20]:
emoji_prodictor("today")

'👌🏼'

# skip-gram model

In [146]:
# Create skip-gram model 
model2 = gensim.models.Word2Vec(docs_snowball, 
                                min_count = 10,  
                                size = 300, 
                                window = 16,
                                sg = 1,
                                workers=8) 

In [41]:
# Save skip-gram model
import pickle
with open('sg_ss.pkl', 'wb') as f:
    pickle.dump(model2, f)

In [125]:
s1 = "let's go grocery and buy some apples"
for w in word_pipeline(s1):
    print(emoji_prodictor(w, model2))


😂
😂
😀
buy
🍎


In [124]:
model2.most_similar("googl",topn=20)

[('now🎮', 0.7763185501098633),
 ('puzzl', 0.7215666770935059),
 ('now~🎮', 0.6516302227973938),
 ('on💡', 0.6338413953781128),
 ('app', 0.6186978816986084),
 ('all😆', 0.5676931738853455),
 ('❗️free❗️', 0.5613607168197632),
 ('now‼️dino', 0.5496898293495178),
 ('microphon', 0.5196519494056702),
 ('go‼️', 0.5183366537094116),
 ('🎰', 0.5135799646377563),
 ('‼️brand', 0.508499026298523),
 ('store', 0.5015548467636108),
 ('iiv', 0.47544899582862854),
 ('available‼️', 0.4744521677494049),
 ('🙋', 0.45962515473365784),
 ('😎✨', 0.4565337300300598),
 ('star', 0.4419368803501129),
 ('apk', 0.4328591525554657),
 ('dailymot', 0.4285903573036194)]

-----------------
validation set

In [18]:
val_df = pd.read_csv("data/emoji_val.2csv",index_col=0)
emoji_set = val_df["emoji"].values.tolist()

In [34]:
val_df["full_desc"] = val_df.full_desc.apply(lambda x: x.strip('][').split(', '))
val_df["desc2"] = val_df.desc2.apply(lambda x: x.strip('][').split(', ') if isinstance(x,str) else "")

In [36]:
for i in val_df.index:
    for j in range(len(val_df.full_desc[i])):
#         print(i)
        val_df.full_desc[i][j] = val_df.full_desc[i][j].strip("/'")
    if val_df.desc2[i]:
        for k in range(len(val_df.desc2[i])):
    #         print(i)
            val_df.desc2[i][k] = val_df.desc2[i][k].strip("/'")

In [38]:
val_df["desc2"][0][1]

'grin'

In [40]:
# build validation set
val_dict = {}
lst = []
for i in val_df.index.tolist():
    lst.extend(val_df.full_desc[i])
    lst.extend(val_df.desc2[i])

In [41]:
vals = list(set(lst))
for val in vals:
    val_dict[val] = []
    for index in val_df.index:
        
        if val in val_df.full_desc[index]:
            val_dict[val].append(val_df.emoji[index])
        elif val in val_df.desc2[index]:
            val_dict[val].append(val_df.emoji[index])


In [ ]:


# with open('val_dict.pickle', 'wb') as handle:
#     pickle.dump(val_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
## update validation set
# val_dict2 = dict();
# for key, value in val_dict.items():
#     if re.split('[- ]', key) == 1:
#         a = key.strip('[():“”]')
#         if a not in val_dict2:
#             val_dict2[a] = value
#         else:
#             val_dict2[a].append(value)
#     else:
#         for i in re.split('[- ]', key):
#             i2 = i.strip('[():“”]')
#             if i2 not in val_dict2:
#                 val_dict2[i2] = value
#             else:
#                 val_dict2[i2].append(value)
            

In [46]:
# with open('val_dict2.pickle', 'wb') as handle:
#     pickle.dump(val_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [17]:
with open('val_dict.pickle', 'rb') as handle:
    val_dict = pickle.load(handle)

# print val_dict == bb

In [69]:
def word2vec_score(val_dict, model):
    total_words = 0
    total_predicts = 0
    correct = 0
    words = []
    word_vectors = model.wv
    for word in val_dict:
#         print(word)
        if word_pipeline(word):
            w = word_pipeline(word)[0]
            words.append(w)
            if w in word_vectors:
                total_words += 1
                if emoji_prodictor(w, model) in val_dict[word]:
                    correct += 1
                    total_predicts += 1
                elif emoji_prodictor(w, model) in emoji_set:
                    total_predicts += 1
    return correct/total_words, correct/total_predicts, words
            

In [147]:
score1 = word2vec_score(val_dict, model1)

In [148]:
score2 = word2vec_score(val_dict, model2)

In [123]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16 and new notes
score1[0:2],score2[0:2]

((0.16936488169364883, 0.19470293486041518),
 (0.22478206724782068, 0.2565742714996446))

In [104]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 16
score1[0:2],score2[0:2]

((0.16084788029925187, 0.19068736141906872),
 (0.21571072319201995, 0.25181950509461426))

In [82]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 20
score1[0:2],score2[0:2]

((0.15149625935162095, 0.17711370262390672),
 (0.21571072319201995, 0.24162011173184358))

In [77]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 150, window = 10
score1[0:2],score2[0:2]

((0.1589775561097257, 0.19480519480519481),
 (0.18952618453865336, 0.22943396226415094))

In [72]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 300, window = 10
score1[0:2],score2[0:2]

((0.17206982543640897, 0.21052631578947367),
 (0.2013715710723192, 0.2492283950617284))

In [66]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 10
score1[0:2],score2[0:2]

((0.16521197007481297, 0.1990984222389181),
 (0.20199501246882792, 0.2577565632458234))

In [61]:
# score for CBOW and skipgram for SS WITH min_count = 5,  size = 500, window = 8
score1[0:2],score2[0:2]

((0.16390532544378697, 0.22124600638977635),
 (0.17810650887573964, 0.2872137404580153))

In [55]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 8
score1[0:2],score2[0:2]

((0.16209476309226933, 0.19667170953101362),
 (0.19887780548628428, 0.2570507655116841))

In [50]:
# score for CBOW and skipgram for SS WITH min_count = 10,  size = 500, window = 3
score1[0:2],score2[0:2]

((0.14526184538653367, 0.19082719082719082),
 (0.1502493765586035, 0.21929026387625114))

In [56]:
# # score for CBOW and skipgram for small sample with SnowballStemmer
# score1[0:2],score2[0:2]

((0.14964788732394366, 0.18695014662756598),
 (0.15307262569832403, 0.23539518900343642))

In [150]:
model2.most_similar('😀',topn=20)

[('👍', 0.5708750486373901),
 ('😉', 0.5507802963256836),
 ('😁', 0.5429814457893372),
 ('⭐️i', 0.5380228757858276),
 ('awesom', 0.5341282486915588),
 ('😅', 0.5143481492996216),
 ('🤗', 0.5049486756324768),
 ('😏', 0.503909707069397),
 ('😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂😂', 0.4923427402973175),
 ('😘', 0.4906120002269745),
 ('wonder', 0.48665982484817505),
 ('goan', 0.4851222038269043),
 ('jeanett', 0.47811296582221985),
 ('skyrim', 0.4774525463581085),
 ('🙀🙀🙀', 0.47732964158058167),
 ('😊', 0.47706007957458496),
 ('👍👊', 0.4713195264339447),
 ('carsss', 0.47071343660354614),
 ('😍thank', 0.4696727693080902),
 ('💞🌺💕', 0.4684681296348572)]

In [97]:
emoji_prodictor('happi', model2)

False

In [98]:
val_dict['happy']

['😀',
 '😀',
 '😃',
 '😃',
 '😄',
 '😄',
 '😁',
 '😁',
 '😆',
 '😆',
 '😅',
 '😅',
 '🤣',
 '🤣',
 '😂',
 '😂',
 '🙂',
 '🙂',
 '🙃',
 '🙃',
 '😉',
 '😉',
 '😊',
 '😊',
 '😇',
 '😍',
 '😍',
 '😘',
 '😘',
 '😗',
 '☺️',
 '☺️',
 '😚',
 '😚',
 '😙',
 '😋',
 '😋',
 '😛',
 '😜',
 '😜',
 '😝',
 '😝',
 '🤑',
 '🤑',
 '🤗',
 '🤗',
 '😏',
 '😏',
 '😌',
 '😌',
 '🤤',
 '🤠',
 '😎',
 '😎',
 '🤓',
 '🤓',
 '😮',
 '😤',
 '😤',
 '🤡',
 '🤡',
 '🙋',
 '🙋🏻',
 [...]]

In [ ]:
def word_pipeline(sentense):
    out = [word for word in sentense.lower().split(" ") if word not in stop]
    out = [snowball.stem(word) for word in out]
    return out

In [115]:
sentense = "under"
[word for word in sentense.lower().split(" ") if word not in stop]

[]

In [116]:
word_pipeline(sentense)model2

[]

In [65]:
emoji_prodictor('us', model1)

'😀'

In [71]:
emoji_prodictor('nyc', model1)

'🚌'

-------------------------
RNN model

In [ ]:
with open('sherlock_homes.txt', 'r') as file:
    text = file.read().lower()
print('text length', len(text))
chars = sorted(list(set(text))) # getting all unique chars
print('total chars: ', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [ ]:
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)
def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss',
                             verbose=1, save_best_only=True,
                             mode='min')

In [ ]:
from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1, min_lr=0.001)
callbacks = [print_callback, checkpoint, reduce_lr]

In [ ]:
model.fit(x, y, batch_size=128, epochs=5, callbacks=callbacks)

In [ ]:
def generate_text(length, diversity):
    # Get random starting text
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [ ]:
print(generate_text(500, 0.2)